In [1]:
import os
from datetime import datetime
# Make logging folder named according to current time
start_time_str = datetime.today().strftime('%d_%m_%Y__%H_%M_%S')
log_dir = './logs/'f"log_{start_time_str}"
os.makedirs(log_dir)

# Initialize logger that adds to text file in current logging folder
from logging_utils import *

init_logger(save_dir=log_dir)
logger = get_logger()




In [2]:
import pathlib
import subprocess
# Get the absolute path to your repository, 
# no matter where you are running this code from

repo_path = os.getcwd() 


git_branch = subprocess.check_output(
    ["git", "-C", repo_path, "rev-parse", "--abbrev-ref", "HEAD"]).strip().decode('UTF-8')

git_commit_short_hash = subprocess.check_output(
    ["git", "-C", repo_path, "describe", "--always"]).strip().decode('UTF-8')


In [3]:
import numpy as np

dim=500
x0=np.squeeze(np.zeros((dim,1)))
x0Type='Zero Vector'

FD_StencilMat=np.zeros((dim,dim))
b = np.ones(dim-1)
c = -2*np.ones(dim)
np.fill_diagonal(FD_StencilMat[1:], b)
np.fill_diagonal(FD_StencilMat[:,1:], b)
np.fill_diagonal(FD_StencilMat, c)
A=FD_StencilMat
AType='1D Laplacian'


nmax_iter = dim
retrain_freq=10
e1 = 1e-8
e2 = 1e-10
nmax_iter = 50
restart   = 64

sigma=0.02
DomainL=-5.0
DomainR=5.0



n_steps =500
InputDim=dim
HiddenDim=dim
HiddenDim2=dim
OutputDim=dim

In [4]:
# Initial log message
logger.info("Benchmark 500 dim problem Using 1 Conv. layer with InputDim channels and InputDim sized kernel. The source is a narrow gaussian that depends on the integer 'time' in a nonlinear way. The gaussian is on the interval [-5,5], and constrained to move on [-4,4]. ")

2020-05-16 01:32:59 [INFO] <ipython-input-4-19e07d14ba94> 2 - Using 1 Conv. layer with InputDim channels and InputDim sized kernel. The source is a narrow gaussian that depends on the integer 'time' in a nonlinear way. The gaussian is on the interval [-5,5], and constrained to move on [-4,4]. 


In [5]:
# Set up parameter class
class MyParamsClass():
    def __init__(self, AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash):
        self.AType = AType
        self.dim = dim
        self.nmax_iter = nmax_iter
        self.restart = restart
        self.n_steps =n_steps
        self.retrain_freq=retrain_freq
        self.e1=e1
        self.e2=e2
        self.x0Type=x0Type
        self.InputDim=InputDim
        self.HiddenDim=HiddenDim
        self.HiddenDim2=HiddenDim2
        self.OutputDim=OutputDim
        self.sigma=sigma
        self.DomainL=DomainL
        self.DomainR=DomainR
        self.git_branch=git_branch
        self.git_commit_short_hash=git_commit_short_hash

In [6]:
import os
import json

params = MyParamsClass(AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash)
sorted_params_dict = {k: params.__dict__[k] for k 
                      in sorted(params.__dict__.keys())}

params_filepath = log_dir+'/params.json'
json.dump(sorted_params_dict, open(params_filepath, 'w'), indent=4)

In [7]:
from src_dir import *

In [8]:
# @nn_preconditioner(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
# def MLGMRES(A, b, x0, e, nmax_iter,IterErr0_sum,IterErr0,ProbCount,Add,restart,debug):
#     return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [9]:
@timer
@nn_preconditioner_timed(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
def MLGMRES(A, b, x0, e, nmax_iter,Err_list,IterErr0_AVG,ProbCount,restart,debug,refine):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)
    

In [10]:
@timer
def GMRES_timed(A, b, x0, e, nmax_iter,restart,debug):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [11]:
Err_list=[]
IterErr0_AVG=0.0

debug=True

refine1=False
refine2=True

run_time=0.0
run_time_ML=0.0
trainTime_total=0.0
forwardTime_Total=0.0
run_time_ML_NoAdd=0.0


x=np.linspace(DomainL,DomainR,dim)



for ProbIdx in range(n_steps):

    loc=4.0*np.sin(0.5*ProbIdx*np.abs(np.cos(0.5*ProbIdx)))
    b=Gauss_pdf(x,loc,sigma)
    b_norm=np.linalg.norm(b)



    OutList1,run_time1_ML=MLGMRES(A, b/b_norm, x0, e1, nmax_iter,Err_list,IterErr0_AVG,ProbIdx,restart,debug,refine1)  # normalize b for optimal NN performance.
    Out=OutList1[0]
    Err_list=OutList1[1]
    IterErr0_AVG=OutList1[2]
    forwardTime1=OutList1[4]


    OutList2,run_time2_ML=MLGMRES(A, b/b_norm, Out[-10], e2, nmax_iter,Err_list,IterErr0_AVG,ProbIdx,restart,debug,refine2)
    Out2=OutList2[0]
    Err_list=OutList2[1]
    IterErr0_AVG=OutList2[2]
    trainTime=OutList2[3]
    forwardTime2=OutList2[4]

    forwardTime_Total=forwardTime1+forwardTime2+forwardTime_Total
    trainTime_total=trainTime+trainTime_total
    run_time_ML=run_time1_ML+run_time2_ML+run_time_ML




    NonML_Out1,run_time1=GMRES_timed(A, b/b_norm, x0, e1, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    NonML_Out2,run_time2=GMRES_timed(A, b/b_norm, NonML_Out1[-10], e2, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    run_time=run_time1+run_time2+run_time

    print(ProbIdx)


logger.info("Runtime of Non-decorated version is:")
logger.info(run_time)

logger.info("Runtime of MLGMRES decorator is:")
logger.info(run_time_ML)


logger.info("Runtime of training (backprop) is:")
logger.info(trainTime_total)

logger.info("Runtime of forward function is:")
logger.info(forwardTime_Total)


 2476
loss: 108115.890625 2477
loss: 108104.953125 2478
loss: 108094.0078125 2479
loss: 108083.078125 2480
loss: 108072.140625 2481
loss: 108061.203125 2482
loss: 108050.2890625 2483
loss: 108039.359375 2484
loss: 108028.421875 2485
loss: 108017.5 2486
loss: 108006.5703125 2487
loss: 107995.6484375 2488
loss: 107984.7421875 2489
loss: 107973.8203125 2490
loss: 107962.90625 2491
loss: 107951.9921875 2492
loss: 107941.078125 2493
loss: 107930.171875 2494
loss: 107919.2578125 2495
loss: 107908.34375 2496
loss: 107897.4375 2497
loss: 107886.53125 2498
loss: 107875.640625 2499
loss: 107864.7265625 2500
loss: 107853.8359375 2501
loss: 107842.9296875 2502
loss: 107832.046875 2503
loss: 107821.140625 2504
loss: 107810.25 2505
loss: 107799.3515625 2506
loss: 107788.46875 2507
loss: 107777.5859375 2508
loss: 107766.6953125 2509
loss: 107755.8125 2510
loss: 107744.9296875 2511
loss: 107734.046875 2512
loss: 107723.171875 2513
loss: 107712.2890625 2514
loss: 107701.4140625 2515
loss: 107690.546875

KeyboardInterrupt: 